# Notebook to fetch blocks from Mode Chain, since the block 1 until the last one.

In [2]:
from web3 import Web3
import pandas as pd
import concurrent.futures
from tqdm import tqdm
import duckdb

In [ ]:
web = Web3(Web3.HTTPProvider("https://mainnet.mode.network/"))

In [ ]:
block_range = list(range(1, int(web.eth.getBlock('latest')['number']) + 1))

In [ ]:
blocks_data_list = []

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    futures = []
    progress_bar = tqdm(total=len(block_range), desc="Getting blocks", unit="request")
    for block_number in block_range:
        futures.append(executor.submit(web.eth.getBlock, block_number))
        
    for future in concurrent.futures.as_completed(futures):
        
        try:
            block = future.result()
            block_data = {
                'block_number': block['number'],
                'block_timestamp': block['timestamp']
            }
            blocks_data_list.append(block_data)
        except Exception as e:
            print(f"Error: {e}")
        finally:
            progress_bar.update(1)
    progress_bar.close()

In [ ]:
blocks = pd.DataFrame(blocks_data_list).sort_values('block_number').reset_index(drop=True)

In [ ]:
blocks.head()

In [ ]:
blocks_range_number = blocks['block_number'].tolist()

In [ ]:
# Get the missing blocks from the range of blocks we requested, sometimes the node is not able to return the data
missing_blocks = list(set(block_range) - set(blocks_range_number))

In [ ]:
for missing_block in missing_blocks:
    block = web.eth.getBlock(missing_block)
    block_data = {
        'block_number': block['number'],
        'block_timestamp': block['timestamp']
    }
    blocks = blocks.append(block_data, ignore_index=True)

In [ ]:
blocks.to_parquet('blocks.parquet')

In [3]:
blocks = pd.read_parquet('../data/mode_chain_blocks.parquet')

In [13]:
blocks['datetime'] = pd.to_datetime(blocks['block_timestamp'], unit='s')

In [14]:
blocks.head()

,block_number,block_timestamp,datetime
0,1,1700167585,2023-11-16 20:46:25
1,2,1700167587,2023-11-16 20:46:27
2,3,1700167589,2023-11-16 20:46:29
3,4,1700167591,2023-11-16 20:46:31
4,5,1700167593,2023-11-16 20:46:33


In [19]:
# Using DuckDB to execute a SQL query using the DataFrame and get the daily blocks
daily_blocks = duckdb.query("""
    WITH daily_blocks AS (
        SELECT
                extract(year from datetime) AS year,
                extract(month from datetime) AS month,
                extract(day from datetime) AS day,
                max(block_number) AS last_block_of_the_day,
                max(block_timestamp) AS last_block_timestamp_of_the_day
        FROM blocks
        GROUP BY 1, 2, 3
        ORDER BY 1, 2, 3 ASC
    )
    
    SELECT cast(concat(year, '-', month, '-', day) as date) AS date, last_block_of_the_day, last_block_timestamp_of_the_day FROM daily_blocks
""").fetchdf()

In [21]:
daily_blocks.head()

,date,last_block_of_the_day,last_block_timestamp_of_the_day
0,2023-11-16,5808,1700179199
1,2023-11-17,49008,1700265599
2,2023-11-18,92208,1700351999
3,2023-11-19,135408,1700438399
4,2023-11-20,178608,1700524799


In [22]:
daily_blocks.to_parquet('../data/mode_chain_daily_blocks.parquet')